# Программирование на Python

*Алла Тамбовцева, НИУ ВШЭ*

## Практикум 7. Работа с API ВКонтакте: собираем посты со стены

Загружаем модули и библиотеки, необходимые для работы:

In [1]:
import requests
import time
import pandas as pd

Для начала давайте посмотрим на документацию API и посмотрим, как к нему формировать запросы: https://dev.vk.com/api/api-requests.

В прошлый раз по инструкции мы получили доступ к API, вспомним шаги.

In [2]:
# вводим id своего приложения
# и проходим по ссылке с этим id

app_id = input("Enter your client id: ")
url = f"https://oauth.vk.com/authorize?client_id={app_id}&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token"
print(url)

Enter your client id: 8186310
https://oauth.vk.com/authorize?client_id=8186310&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token


In [3]:
# копируем токен доступа

token = input("Enter your token here: ")

Enter your token here: vk1.a.7IiTv6i2KUG7RtBNf_fOvGOk1TLzV3Sq0pBNeVphPnxAh0D-jTCugSP2K8hmLkxxW-RZlntlGuvpVLBy_otT0uCH9gu9CV21tZ07dguH2h2bz5Zl6SWq1sm1ZjiNXsIu2GcmfSCUQUAnIj7yP7_BRjH4LeeuD4E4Nn3G7-E9k0KG7wl8c0IOoxobIN59f3L9


На этом практическом занятии мы будем выгружать посты из сообщества [Цитатник ВШЭ](https://vk.com/hseteachers). Сохраним в переменные версию API, ссылку для метода работы со стеной сообщества и название сообщества:

In [4]:
v = "5.131"
main_wall = "https://api.vk.com/method/wall.get"
domain = "hseteachers"

Функция `get()` из библиотеки `requests` умеет подставлять в запрос необходимые параметры и объединять их с помощью `?` и `&`. Сохраним необходимые параметры в виде словаря:

In [5]:
params_wall = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v}

А теперь сформируем запрос и выгрузим результаты в формате JSON – в Python данные в таком формате будут представлены в виде словаря:

In [6]:
req_wall = requests.get(main_wall, params = params_wall)

In [7]:
json_wall = req_wall.json()

Извлечём из этого большого словаря элемент, который хранит непосредственно результаты – список из маленьких словарей с информацией о постах (1 словарь = 1 пост):

In [ ]:
items_wall = json_wall['response']['items']

Посмотрим на один элемент такого списка:

In [ ]:
i = items_wall[0]
i

Поработаем с ним!

### Задача 1

Извлечь из элемента `i` следующие компоненты:

* id поста;
* дата поста;
* текст поста;
* число лайков;
* число репостов;
* число просмотров;
* число комментариев.

In [ ]:
print(i["id"], i["date"], i["text"])
print(i["likes"]["count"], i["reposts"]["count"], i["views"]["count"],
      i["comments"]["count"])

### Задача 2

Напишите функцию `get_posts()`, которая принимает на вход словарь, аналогичный сохранённому в `i`, и возвращает список из следующих характеристик:

* id поста;
* дата поста;
* текст поста;
* число лайков;
* число репостов;
* число просмотров;
* число комментариев.

In [ ]:
# добавляем ловлю исключений
# на случай, если какие-то характеристики будут отсутствовать
# по-хорошему надо на каждую характеристику поставить свой try-except,
# тут более грубое решение, просто для примера

def get_posts(i):
    try:
        id_ = i["id"]
        date = i["date"]
        text = i["text"]
        likes = i["likes"]["count"]
        repos = i["reposts"]["count"]
        views = i["views"]["count"]
        comments = i["comments"]["count"]
    except:
        id_ = i["id"]
        date = i["date"]
        text = ""
        likes = -100
        repos = -100
        views = -100
        comments = -100
    L = [id_, date, text, likes, repos, views, comments] 
    return L

### Задача 3

Примените функцию `get_posts()` ко всем элементам списка `items_wall` и сохраните полученные результаты в список `posts`. 

In [ ]:
posts = []

for i in items_wall:
    p = get_posts(i)
    posts.append(p) 

### Задача 4

Прочитайте в документации к API ВКонтакте про аргумент `offset` в методе `wall.get`. Используя полученную информацию и блоки кода ниже, выгрузите и сохраните в список `items_more` данные ещё по 9300 постам на стене сообщества.

**Подсказка:** чтобы расширять список правильным образом, используйте метод `.extend()`, а не `.append()`, он добавляет не один элемент, а сразу несколько.

In [ ]:
params_wall_long = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "offset" : 100,
              "v" : v} 

In [ ]:
items_more = []  

for i in range(92):
    req_wall_long = requests.get(main_wall, params = params_wall_long)
    json_wall_long = req_wall_long.json()
    items_wall_long = json_wall_long['response']['items']
    items_more.extend(items_wall_long) 
    params_wall_long["offset"] = params_wall_long["offset"] + 100
    time.sleep(1.5)

Теперь извлечём из каждого элемента `items_more` нужную информацию и расширим список `posts`, который у нас уже был до этого:

In [ ]:
for i in items_more:
    p = get_posts(i)
    posts.append(p)

In [ ]:
len(posts)  # все идёт по плану

In [ ]:
posts[202]

Преобразуем результат в датафрейм, добавим названия столбцов:

In [ ]:
dat = pd.DataFrame(posts)
dat.columns = ["id", "timestamp", "post", "likes", 
               "reposts", "views", "comments"]

Несколько строк датафрейма для примера:

In [ ]:
dat.head()

Разобьём текст поста по `#`, чтобы извлечь тэги:

In [ ]:
dat["post"][10] 

In [ ]:
with_tags = dat["post"].str.split("#", expand = True)
with_tags

Основная информация – это первые два тэга, имя преподавателя и курс (по крайней мере, в большинстве случаев это так). Заберём для дальнейшей работы только их:

In [ ]:
small = with_tags.loc[:, 0:2] 
small.columns = ["text", "teacher", "course"] 

Склеим датафрейм `dat` с датафреймом `small` по столбцам:

In [ ]:
final = pd.concat([dat, small], axis = 1) 

Заполним пропуски – добавим «пустой» текст в ячейки, где нет никаких значений:

In [ ]:
final = final.fillna("") 

Избавимся от лишних пробелов и отступов в текстовых данных:

In [ ]:
final["text"] = final["text"].apply(lambda x: x.strip())
final["teacher"] = final["teacher"].apply(lambda x: x.strip())
final["course"] = final["course"].apply(lambda x: x.strip())

Осталось поработать с форматом времени в столбце `timestamp`.

In [ ]:
final["timestamp"]

In [ ]:
t = final["timestamp"][0]
t

Импортируем из модуля `datetime` функцию `datetime`, она поможет нам получить дату и время в привычном формате:

In [ ]:
from datetime import datetime

In [ ]:
datetime.utcfromtimestamp(t)

In [ ]:
datetime.utcfromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')

Напишем функцию для преобразования временной метки:

In [ ]:
def time_transform(t):
    r = datetime.fromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')
    return r

Применим её ко всем элементам столбца:

In [ ]:
final["datetime"] = final["timestamp"].apply(time_transform)

In [ ]:
final.head() 

Теперь можем разбить дату-время по пробелу, чтобы получить отдельные столбцы с датой и временем (механизм нам уже известен, мы разбивали пост по `#` выше):

In [ ]:
dt = final["datetime"].str.split(" ", expand = True)
dt.columns = ["date", "time"] 

In [ ]:
final2 = pd.concat([final, dt], axis = 1)

In [ ]:
final2

In [ ]:
final2.to_excel("posts.xlsx")

In [ ]:
final2[final2["course"] == "Психология"] 

In [ ]:
final2[final2["teacher"].str.contains("тамб")] 